# Treinamento do Chatbot

Para começar, é necessário que você tenha uma versão recente do Python e o pip instalados no seu computador, logo mais, vamos instalar as seguintes bibliotecas:

- NLTK — É uma das ferramentas mais utilizadas para processamento de linguagem natural, foi desenvolvida em Python e tem uma gama muito grande de recursos, como: classificação, tokenização, stemming, tagging, parsing e raciocínio semântico. Todas essas funções são utilizadas para análise de texto;
- Numpy — É uma biblioteca para a linguagem Python com funções para se trabalhar com computação numérica, e que pode realizar operações de álgebra linear de maneira muito eficiente;
- Tensorflow — É uma biblioteca de código aberto criada para aprendizado de máquina, computação numérica e muitas outras tarefas;
- Keras — Por último, e de extrema importância, usamos o Keras para a estrutura de aprendizado profundo, essa lib poderosíssima é uma das principais APIs de redes neurais de alto nível.

In [ ]:
!pip install -r requirements.txt

Agora, vamos criar o arquivo train.py, onde vamos ter o código para ler os dados de linguagem natural e usar a rede neural sequencial keras para criar nosso modelo.
Nesse código vamos dividir a explicação em algumas partes, para facilitar o entendimento.
Realizamos a importação e configuração inicial das libs que iremos utilizar:

In [ ]:
import os
import json
import pickle
import nltk
import random
import numpy as np

from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

if not os.path.exists('models'):
    os.makedirs('models')

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

Inicializamos a nossa lista de palavras, classes, documentos e definimos quais palavras serão ignoradas, percorremos a nossa lista de intenções, que foram lidas pelo código e com ajuda do nltk fazemos a tokenização dos patterns e adicionamos na lista de palavras, adicionamos também aos documentos para termos a identificação da tag para cada palavra e adicionamos as tags a nossa lista de classe:

In [ ]:
# inicializaremos nossa lista de palavras, classes, documentos e 
# definimos quais palavras serão ignoradas
words = []
documents = []
intents = json.loads(open('data/intents.json').read())

In [ ]:
# adicionamos as tags em nossa lista de classes
classes = [i['tag'] for i in intents['intents']]
ignore_words = ["!", "@", "#", "$", "%", "*", "?"]

In [ ]:
# percorremos nosso array de objetos
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # com ajuda no nltk fazemos aqui a tokenizaçao dos patterns 
        # e adicionamos na lista de palavras
        word = nltk.word_tokenize(pattern)
        words.extend(word)

        # adiciona aos documentos para identificarmos a tag para a mesma
        documents.append((word, intent['tag']))

Em seguida, vamos lematizar, ou seja, transformar as palavras em seus significados básicos, com o objetivo de restringir tudo ao nível mais simples possível.
Um exemplo de lematização ocorre com verbos, tipo, escrevendo, escreveu e escreve tem o mesmo lema que é escrever.
Logo, classificamos nossas listas e estamos prontos para construir o modelo de aprendizado profundo.

In [ ]:
# lematizamos as palavras ignorando os palavras da lista ignore_words
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

In [ ]:
# classificamos nossas listas
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [ ]:
# salvamos as palavras e classes nos arquivos pkl
pickle.dump(words, open('models/words.pkl', 'wb'))
pickle.dump(classes, open('models/classes.pkl', 'wb'))

Vamos começar nosso treinamento criando um array vazio de treinamento e criando uma lista de saídas vazias de acordo com o tamanho das nossas classes.
Percorremos nossos documentos, inicializamos um array de bag vazio, inserimos no nosso pattern_word a nossa palavra correspondente àquele padrão, lematizamos cada uma delas na tentativa de representar palavras relacionadas, inserimos 1 no bag se a correspondência de palavras for encontrada no pattern atual e utilizamos o output_row como uma chave para a lista, onde a saída será 0 para cada tag e 1 para a tag atual.
Após isso embaralhamos nosso conjunto de treinamentos, transformamos em numpy array e definimos uma lista de treinos, sendo x os patterns e y as intenções:

In [ ]:
# inicializamos o treinamento
training = []
output_empty = [0] * len(classes)
for document in documents:
    # inicializamos o saco de palavras 
    bag = []

    # listamos as palavras do pattern
    pattern_words = document[0]

    # lematizamos cada palavra 
    # na tentativa de representar palavras relacionadas
    pattern_words = [lemmatizer.lemmatize( word.lower()) for word in pattern_words]

    # criamos nosso conjunto de palavras com 1, 
    # se a correspondência de palavras for encontrada no padrão     atual
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    # output_row atuará como uma chave para a lista, 
    # onde a saida será 0 para cada tag e 1 para a tag atual
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [ ]:
# embaralhamos nosso conjunto de treinamentos e transformamos em numpy array
random.shuffle(training)
training = np.array(training)

In [ ]:
# criamos lista de treino sendo x os patterns e y as intenções
x = list(training[:, 0])
y = list(training[:, 1])

Com nossos dados de treinamento prontos, usaremos o modelo de aprendizado profundo keras chamado sequencial, esse modelo sequencial é uma das redes neurais mais simples, um perceptron multicamadas, que em particular tem 3 camadas, com a primeira tendo 128 neurônios, a segunda 64 e a terceira tendo o número de intenções igual o número de neurônios, o objetivo dessa rede é tentar prever qual base escolher de acordo com alguns dados.
Esse modelo será treinado com descida gradiente estocástica que é um tópico beeeem complexo, mas que tem muito conteúdo no senhor google e no link disponibilizado.
Depois que nosso modelo é treinado será salvo no model.h5 como numpy array e é com esse modelo que vamos criar nossa GUI do chatbot.

In [ ]:
# Criamos nosso modelo com 3 camadas. 
# Primeira camada de 128 neurônios, 
# segunda camada de 64 neurônios e terceira camada de saída 
# contém número de neurônios igual ao número de intenções para prever a intenção de saída com softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y[0]), activation='softmax'))
model.summary()

In [ ]:
# O modelo é compilado com descida de gradiente estocástica 
# com gradiente acelerado de Nesterov.
# A ideia da otimização do Momentum de Nesterov, ou Nesterov Accelerated Gradient (NAG), 
# é medir o gradiente da função de custo não na posição local,
# mas ligeiramente à frente na direção do momentum. 
# A única diferença entre a otimização de Momentum é que o gradiente é medido em θ + βm em vez de em θ.
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])

In [ ]:
# ajustamos e salvamos o modelo
m = model.fit(np.array(x), np.array(y), epochs=200, batch_size=5, verbose=1)
model.save('models/model.h5', m)